In [7]:
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
import json
from PIL import Image 
import torch
from librosa.filters import mel as librosa_mel_fn
import msynth.efficientnet_pytorch.model as modeleff
from torch.autograd import Variable
import torchaudio 
import einops 
import os
import soundfile as sf

OSError: libc10_cuda.so: cannot open shared object file: No such file or directory

In [ ]:
def sampler(controls, samples): #(16,1,4)
    sampled = torch.bmm(controls,
            samples)    
    synthesized_audio = einops.rearrange(sampled, 'b d s -> d (b s)')#(1,465696)
    return synthesized_audio

def get_melspec(audio):
    stft = torch.stft(audio,n_fft=441*4,hop_length=441*2)
    real_part, imag_part = stft.unbind(-1)
    magnitude_concat = torch.sqrt(real_part ** 2 + imag_part ** 2)
    mel_output = torch.matmul(mel_basis, magnitude_concat.float())
    log_mel_spec_concat = torch.log10(torch.clamp(mel_output, min=1e-5))#(1,128,529)
    return log_mel_spec_concat

In [ ]:
class FunctionWithNumericalGrad(torch.autograd.Function):
    
    @staticmethod
    def forward(cxt, synth_params):
        ctx.save_for_backward(synth_params)
        # do forward process
        synth_signal = sampler(synth_params)

        return synth_signal

    def synth_func(synth_params):

        return synthesized_audio

    @staticmethod
    def backward(cxt, grad_output):

        epsilon = 0.1 

        synth_params = cxt.saved_tensors
  
        grads_synth_params = torch.empty_like(synth_params)

        # generate random vector of +-1s
        delta_k= np.random.randint(0,high=1,size=synth_params.shape)
        delta_k[delta_k==0]=-1
        print(delta_k)
        #delta_k = rademacher(synth_params.shape).numpy()

        params = []
        
        # synth +- forward and backward passes
        J_plus = sampler(synth_params + delta_k)

        J_minus = sampler(synth_params - delta_k)

        grad_synth = J_plus - J_minus

        # loop for each element of synth_params
        params_sublist = []
        for sub_p_idx in range(len(synth_params)):
            grad_s = grad_synth / (2 * epsilon * delta_k[sub_p_idx])
            params.append(np.sum(grad_output * grad_s))

        grad_synth = np.array(synth_params)

        return grad_synth

In [ ]:

model = modeleff.EfficientNet.from_name('efficientnet-b0')
model._change_in_channels(1)
    
list_of_notes = ["47.wav","44.wav","42.wav","40.wav"]
song=["mary.wav"]

ground_truth = torch.tensor([[0,1,0,0],[0,0,1,0],[0,0,0,1],[0,0,1,0],
        [0,1,0,0],[0,1,0,0],[0,1,0,0],[0,1,0,0],
        [0,0,1,0],[0,0,1,0],[0,0,1,0],[0,0,1,0],[1,0,0,0],
        [1,0,0,0],[1,0,0,0],[1,0,0,0]]).type(torch.DoubleTensor)
        
audio_d,sr = torchaudio.load("../data/musicfiles/" + list_of_notes[0])
audio_b,sr = torchaudio.load("../data/musicfiles/" + list_of_notes[1])
audio_a,sr = torchaudio.load("../data/musicfiles/" + list_of_notes[2])
audio_g,sr = torchaudio.load("../data/musicfiles/" + list_of_notes[3])
mary,sr = torchaudio.load("../data/musicfiles/" + song[0])

samples = torch.cat([torch.unsqueeze(audio_d,dim=0),torch.unsqueeze(audio_b,dim=0),
torch.unsqueeze(audio_a,dim=0),torch.unsqueeze(audio_g,dim=0)],dim=0) #(4,1,29106)
samples = samples.permute((1,0,2)).repeat(16,1,1).type(torch.DoubleTensor) #(16,4,29106)
resynthesized_mary = sampler(torch.unsqueeze(ground_truth,dim=1),samples)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

ground_truth = torch.unsqueeze(ground_truth, dim=2) #(16,4,1)

loss = torch.nn.MSELoss()

mel_basis = librosa.filters.mel(44100,n_fft=441*4,n_mels=128)
mel_basis = torch.from_numpy(mel_basis).float()

log_mel_spec_mary = get_melspec(resynthesized_mary.type(torch.FloatTensor))

ground_truth_matrix =ground_truth.permute((0,2,1)) #(16,1,4)

for i in range(500):

    optimizer.zero_grad()
    
    feats = model.extract_features(torch.unsqueeze(log_mel_spec_mary,dim=0))[0] #(4,1,16)
    feats=feats.permute((2,1,0)) #(16,1,4)
    output = torch.nn.functional.sigmoid(feats*10).type(torch.DoubleTensor)

    synthesized_audio = FunctionWithNumericalGrad.apply(output)
    
    #synthesized_audio = sampler(output,samples)
    log_mel_spec_concat = get_melspec(synthesized_audio)

    loss_val = loss(log_mel_spec_concat,log_mel_spec_mary)
    loss_val.backward()

    optimizer.step()

    print("Loss is: ", loss_val)

if not os.path.isdir("reconstruction"):
    os.mkdir("reconstruction")

sf.write("reconstruction/resynthesized_mary.wav", resynthesized_mary.permute(1,0).detach().numpy(), 44100)
sf.write("reconstruction/synthesized_audio.wav", synthesized_audio.permute(1,0).detach().numpy(), 44100)
